# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

X. Zhang  ->  X. Zhang  |  ['X. Zhang']
J. Liu  ->  J. Liu  |  ['J. Liu']
N. Neumayer  ->  N. Neumayer  |  ['N. Neumayer']
J. Liu  ->  J. Liu  |  ['J. Liu']
J. Liu  ->  J. Liu  |  ['J. Liu']
S. Jiao  ->  S. Jiao  |  ['S. Jiao']
K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
E. Matthews  ->  E. Matthews  |  ['E. Matthews']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']


O. Krause  ->  O. Krause  |  ['O. Krause']


Arxiv has 97 new papers today
          8 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/8 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2503.11740


extracting tarball to tmp_2503.11740...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.11778


extracting tarball to tmp_2503.11778...

...

extracting tarball to tmp_2503.12675...

 done.
Retrieving document from  https://arxiv.org/e-print/2503.12707


extracting tarball to tmp_2503.12707... done.


Found 132 bibliographic references in tmp_2503.12707/sn-article.bbl.
Issues with the citations
syntax error in line 12: unbalanced braces


/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:695: LatexWarning: Falling back to regex to find figures
  warnings.warn(LatexWarning("Falling back to regex to find figures"))
/opt/hostedtoolcache/Python/3.9.21/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:698: LatexWarning: Fallback: found 0 figures.
  warnings.warn(LatexWarning("Fallback: found " + str(len(results)) + " figures."))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11856-b31b1b.svg)](https://arxiv.org/abs/2503.11856) | **A spectroscopic map of the Galactic centre -- Observations and resolved stars**  |
|| A. Feldmeier-Krause, et al. -- incl., <mark>N. Neumayer</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *20 pages (+ 7 pages Appendix), 17 (+ 4) figures, accepted A&A*|
|**Abstract**|            The Galactic Centre region contains a dense accumulation of stars, which can be separated into two components: A flattened and dense nuclear star cluster (NSC), and a surrounding, more extended and more flattened, nuclear stellar disc (NSD). Previous studies have collected a few thousand spectra of the inner NSC, and also the outer NSD, and measured line-of-sight velocities and metallicities. Until now, such measurements exist only for a few 100 stars in the region where the stellar surface density transitions from being dominated by the NSC into being dominated by the NSD. We want to study the stellar population from the centre of the NSC out to well beyond its effective radius, where the NSD dominates. We investigate whether and how the mean properties and kinematics of the stars change systematically. We conducted spectroscopic observations with Flamingos-2 in the K-band via a continuous slit-scan. The data extend from the central NSC into the inner NSD, out to 32 pc from Sgr A* along Galactic longitude l. Based on their CO equivalent width, we classify the stars as hot or cool stars. The former are massive, young stars, while almost all of the latter are older than one to a few Gyr. We measure the overall metallicity [M/H] and line-of-sight velocity for >2,500 cool stars, and present the first continuous spatial maps and profiles of the mean value of various stellar and kinematic parameters. We identify hot, young stars across the field of view. Some stars appear to be isolated, while others accumulate near the Quintuplet cluster or the central parsec cluster. The position-velocity curve of the cool stars shows no dependence on [M/H], but it depends on the colour of the stars. The colour may be a tracer of the line-of-sight distance and thus distinguish stars located in the NSC from those in the NSD. [abridged]         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.12707-b31b1b.svg)](https://arxiv.org/abs/2503.12707) | **Advancing European High-Contrast Imaging R&D Towards the Habitable Worlds Observatory**  |
|| I. Laginja, et al. -- incl., <mark>E. Matthews</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>O. Krause</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *Accepted for publication in Astrophysics and Space Science*|
|**Abstract**|            The Habitable Worlds Observatory (HWO) will enable a transformative leap in the direct imaging and characterization of Earth-like exoplanets. For this, NASA is focusing on early investment in technology development prior to mission definition and actively seeking international partnerships earlier than for previous missions. The "R&D for Space-Based HCI in Europe" workshop, held in March 2024 at Paris Observatory, convened leading experts in high-contrast imaging (HCI) to discuss European expertise and explore potential strategies for European contributions to HWO. This paper synthesizes the discussions and outcomes of the workshop, highlighting Europe's critical contributions to past and current HCI efforts, the synergies between ground- and space-based technologies, and the importance of laboratory testbeds and collaborative funding mechanisms. Key conclusions include the need for Europe to invest in technology development for areas such as deformable mirrors and advanced detectors, and establish or enhance laboratory facilities for system-level testing. Putting emphasis on the urgency of aligning with the timeline of the HWO, the participants called on an open affirmation by the European Space Agency (ESA) that a European contribution to HWO is clearly anticipated, to signal national agencies and unlock funding opportunities at the national level. Based on the expertise demonstrated through R&D, Europe is poised to play a pivotal role in advancing global HCI capabilities, contributing to the characterization of temperate exoplanets and fostering innovation across domains.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.12675-b31b1b.svg)](https://arxiv.org/abs/2503.12675) | **Discovery of two new polars evolved past the period bounce**  |
|| T. Cunningham, et al. -- incl., <mark>K. El-Badry</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *18 pages. Submitted to MNRAS*|
|**Abstract**|            We report the discovery of two new magnetic cataclysmic variables with brown dwarf companions and long orbital periods ($P_{\rm orb}=95\pm1$ and $104\pm2$ min). This discovery increases the sample of candidate magnetic period bouncers with confirmed sub-stellar donors from four to six. We also find their X-ray luminosity from archival XMM-Newton observations to be in the range $L_{\rm X}\approx10^{28}$$-$$10^{29} \mathrm{erg\,s^{-1}}$ in the 0.25$-$10 keV band. This low luminosity is comparable with the other candidates, and at least an order of magnitude lower than the X-ray luminosities typically measured in cataclysmic variables. The X-ray fluxes imply mass transfer rates that are much lower than predicted by evolutionary models, even if some of the discrepancy is due to the accretion energy being emitted in other bands, such as via cyclotron emission at infrared wavelengths. Although it is possible that some or all of these systems formed directly as binaries containing a brown dwarf, it is likely that the donor used to be a low-mass star and that the systems followed the evolutionary track for cataclysmic variables, evolving past the period bounce. The donor in long period systems is expected to be a low-mass, cold brown dwarf. This hypothesis is supported by near-infrared photometric observations that constrain the donors in the two systems to be brown dwarfs cooler than $\approx$1100 K (spectral types T5 or later), most likely losing mass via Roche Lobe overflow or winds. The serendipitous discovery of two magnetic period bouncers in the small footprint of the XMM-Newton source catalog implies a large space density of these type of systems, possibly compatible with the prediction of 40$-$70 per cent of magnetic cataclysmic variables to be period bouncers.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11740-b31b1b.svg)](https://arxiv.org/abs/2503.11740) | **Square Kilometre Array Science Data Challenge 3a: foreground removal for an EoR experiment**  |
|| A. Bonaldi, et al. -- incl., <mark>X. Zhang</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *29 pages, 10 figures, submitted to MNRAS*|
|**Abstract**|            We present and analyse the results of the Science data challenge 3a (SDC3a, this https URL), an EoR foreground-removal community-wide exercise organised by the Square Kilometre Array Observatory (SKAO). The challenge ran for 8 months, from March to October 2023. Participants were provided with realistic simulations of SKA-Low data between 106 MHz and 196 MHz, including foreground contamination from extragalactic as well as Galactic emission, instrumental and systematic effects. They were asked to deliver cylindrical power spectra of the EoR signal, cleaned from all corruptions, and the corresponding confidence levels. Here we describe the approaches taken by the 17 teams that completed the challenge, and we assess their performance using different metrics. The challenge results provide a positive outlook on the capabilities of current foreground-mitigation approaches to recover the faint EoR signal from SKA-Low observations. The median error committed in the EoR power spectrum recovery is below the true signal for seven teams, although in some cases there are some significant outliers. The smallest residual overall is $4.2_{-4.2}^{+20} \times 10^{-4}\,\rm{K}^2h^{-3}$cMpc$^{3}$ across all considered scales and frequencies. The estimation of confidence levels provided by the teams is overall less accurate, with the true error being typically under-estimated, sometimes very significantly. The most accurate error bars account for $60 \pm 20$\% of the true errors committed. The challenge results provide a means for all teams to understand and improve their performance. This challenge indicates that the comparison between independent pipelines could be a powerful tool to assess residual biases and improve error estimation.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.11778-b31b1b.svg)](https://arxiv.org/abs/2503.11778) | **The Power of the Cosmic Web**  |
|| J. Sunseri, A. E. Bayer, <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *14 pages, 10 figures*|
|**Abstract**|            We study the cosmological information contained in the cosmic web, categorized as four structure types: nodes, filaments, walls, and voids, using the Quijote simulations and a modified  nexus+ algorithm. We show that splitting the density field by the four structure types and combining the power spectrum in each provides much tighter constraints on cosmological parameters than using the power spectrum without splitting. We show the rich information contained in the cosmic web structures -- related to the Hessian of the density field -- for measuring all of the cosmological parameters, and in particular for constraining neutrino mass. We study the constraints as a function of Fourier scale, configuration space smoothing scale, and the underlying field. For the matter field with $k_{\rm max}=0.5\,h/{\rm Mpc}$, we find a factor of $\times20$ tighter constraints on neutrino mass when using smoothing scales larger than 12.5~Mpc/$h$, and $\times80$ tighter when using smoothing scales down to 1.95~Mpc/$h$. However, for the CDM+Baryon field we observe a more modest $\times1.7$ or $\times3.6$ improvement, for large and small smoothing scales respectively. We release our new python package for identifying cosmic structures pycosmmommf at this https URL to enable future studies of the cosmological information of the cosmic web.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.12410-b31b1b.svg)](https://arxiv.org/abs/2503.12410) | **Searching for accreting compact binary systems from spectroscopy and photometry: Application to LAMOST spectra**  |
|| X. Zhao, S. Wang, <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *18pages, 15 figures, accepted for publication in APJ*|
|**Abstract**|            Compact objects undergoing mass transfer exhibit significant (and double-peaked) $H_{\alpha}$ emission lines. Recently, new methods have been developed to identify black hole X-ray binaries (BHXBs) and calculate their systematic parameters using $H_{\alpha}$ line parameters, such as the full-width at half maximum (FWHM), equivalent width (EW), and separation of double peaks. In addition, the FWHM-EW plane from spectroscopy and the $H_{\alpha}$ color-color diagram from photometry can be used for rapid stellar classification. We measure the $H_{\alpha}$ and $H_{\beta}$ profiles (e.g., FWHM and EW) using the LAMOST DR9 low- and medium-resolution spectra, and calculate the systematic parameters (e.g., velocity semi-amplitude of the donor star, mass ratio, inclination angle, and mass of the accretor). A new correlation between FWHM and $K_{\rm 2}$, $K_{\rm 2} = 0.205(18)\ \rm{FWHM}$, is obtained for cataclysmic variables (CVs) in our sample. Both the FWHM-EW plane and the $H_{\alpha}$ color-color diagram can distinguish CVs with FWHM $\gtrsim$ 1000 km/s from Be stars and young stellar objects (YSOs) to some extent. To improve classification accuracy and enhance the selection of compact objects, we propose a new set of idealized filters with effective widths of 30 Å, 130 Å, and 400 Å\ for the narrow $H_{\alpha}$ filter, broad $H_{\alpha}$ filter, and $r$-band filter, respectively.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.12449-b31b1b.svg)](https://arxiv.org/abs/2503.12449) | **Calibration of Complementary Metal-oxide-semiconductor Sensor-based Photometry to a Few-millimagnitude Precision: The Case of the Mini-SiTian Array**  |
|| K. Xiao, et al. -- incl., <mark>J. Liu</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *8 pages, 6 figures, ApJL accepted, see main results in Figures 4*|
|**Abstract**|            We present a pioneering achievement in the high-precision photometric calibration of CMOS-based photometry, by application of the Gaia BP/RP (XP) spectra-based synthetic photometry (XPSP) method to the mini-SiTian array (MST) photometry. Through 79 repeated observations of the $\texttt{f02}$ field on the night, we find good internal consistency in the calibrated MST $G_{\rm MST}$-band magnitudes for relatively bright stars, with a precision of about 4\,mmag for $G_{\rm MST}\sim 13$. Results from more than 30 different nights (over 3100 observations) further confirm this internal consistency, indicating that the 4\,mmag precision is stable and achievable over timescales of months. An independent external validation using spectroscopic data from the Large Sky Area Multi-Object Fiber Spectroscopic Telescope (LAMOST) DR10 and high-precision photometric data using CCDs from Gaia DR3 reveals a zero-point consistency better than 1\,mmag. Our results clearly demonstrate that CMOS photometry is on par with CCD photometry for high-precision results, highlighting the significant capabilities of CMOS cameras in astronomical observations, especially for large-scale telescope survey arrays.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2503.12513-b31b1b.svg)](https://arxiv.org/abs/2503.12513) | **The HASHTAG project II. Giant molecular cloud properties across the M31 disc**  |
|| Y. Deng, et al. -- incl., <mark>S. Jiao</mark> |
|*Appeared on*| *2025-03-18*|
|*Comments*| *18 pages, 13 figures, accepted for publication in MNRAS*|
|**Abstract**|            We present a study of giant molecular cloud (GMC) properties in the Andromeda galaxy (M31) using CO(3-2) data from the James Clerk Maxwell Telescope (JCMT) in selected regions across the disc and in the nuclear ring, and comparing them with CO(1-0) observations from the IRAM 30m telescope in the same regions. We find that GMCs in the centre of M31 generally exhibit larger velocity dispersions ($\sigma$) and sizes ($R$) compared to those in the disc, while their average surface density ($\Sigma$) and turbulent pressure ($P_{\rm turb}$) are lower. This low turbulent pressure in the central region is primarily due to the low density of molecular gas. The estimated GMC properties depend on the choice of CO transitions. Compared to CO(1-0), CO(3-2) exhibits smaller velocity dispersion and equivalent radius but higher surface density. These differences highlight the distinct physical conditions probed by different molecular gas tracers. We estimate the virial parameter $\alpha_{\rm vir}\propto \sigma^2 R/\Sigma$ and find that most molecular clouds exhibit high values ($\alpha_{\rm vir} \sim 4-6$) for both CO transitions, indicating that they are unbound. Furthermore, clouds in the nuclear ring display even larger $\alpha_{\rm vir}$ values of $\lesssim 100$, suggesting that they may be highly dynamic, short-lived structures, although they could potentially achieve equilibrium under the external pressure exerted by the surrounding interstellar medium.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2503.11856/./mhprofile_running_noex_cutnorth_cutblue.png', 'tmp_2503.11856/./lmetprof_running_noex_cutnorth_cutblue.png', 'tmp_2503.11856/./mh_fu_hk_noex.png', 'tmp_2503.11856/./SPar_bin240__noex1.png', 'tmp_2503.11856/./Kcol-Ak_bin40__noex1.png']
copying  tmp_2503.11856/./mhprofile_running_noex_cutnorth_cutblue.png to _build/html/
copying  tmp_2503.11856/./lmetprof_running_noex_cutnorth_cutblue.png to _build/html/
copying  tmp_2503.11856/./mh_fu_hk_noex.png to _build/html/
copying  tmp_2503.11856/./SPar_bin240__noex1.png to _build/html/
copying  tmp_2503.11856/./Kcol-Ak_bin40__noex1.png to _build/html/
exported in  _build/html/2503.11856.md
    + _build/html/tmp_2503.11856/./mhprofile_running_noex_cutnorth_cutblue.png
    + _build/html/tmp_2503.11856/./lmetprof_running_noex_cutnorth_cutblue.png
    + _build/html/tmp_2503.11856/./mh_fu_hk_noex.png
    + _build/html/tmp_2503.11856/./SPar_bin240__noex1.png
    + _build/html/tmp_2503.11856/./Kcol-Ak_bin40__noex1.png
fo

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\ensuremath{\mu}m}$
$\newcommand{\angstrom}{\mathring{A}}$
$\newcommand{\teff}{\ensuremath{T_\textrm{eff}}\xspace}$
$\newcommand{\re}{\ensuremath{R_\textrm{e}}\xspace}$
$\newcommand{\logg}{\ensuremath{\log(\rm g)}\xspace}$
$\newcommand{\mh}{[M/H]\xspace}$
$\newcommand{\kms}{km s\ensuremath{^{-1}}\xspace}$
$\newcommand{\msun}{\ensuremath{M_{\sun}}\xspace}$
$\newcommand{\vlos}{\ensuremath{V_\textrm{LOS}}\xspace}$
$\newcommand{\dlos}{\ensuremath{d_\textrm{LOS}}\xspace}$
$\newcommand{\slos}{\ensuremath{\sigma_{\textrm{LOS}}}\xspace}$
$\newcommand{\sgra}{Sgr~A^\star\xspace}$
$\newcommand{\sgr}{Sgr~A^\star\xspace}$
$\newcommand{\col}{\ensuremath{H-K_{S}}\xspace}$
$\newcommand{\ak}{\ensuremath{A_{K_S}}\xspace}$
$\newcommand{\thetable}{E.\arabic{table}}$</div>



<div id="title">

# A spectroscopic map of the Galactic centre: Observations and resolved stars 

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.11856-b31b1b.svg)](https://arxiv.org/abs/2503.11856)<mark>Appeared on: 2025-03-18</mark> -  _20 pages (+ 7 pages Appendix), 17 (+ 4) figures, accepted A&A_

</div>
<div id="authors">

A. Feldmeier-Krause, et al. -- incl., <mark>N. Neumayer</mark>

</div>
<div id="abstract">

**Abstract:** The Galactic Centre region contains a dense accumulation of stars, which can be separated into two components: A mildly flattened and extremely dense nuclear star cluster (NSC), and a surrounding, more extended and more flattened, nuclear stellar disc (NSD).  Previous studies have collected a few thousand spectra of the inner NSC, and also the outer NSD, and measured line-of-sight velocities and metallicities. Until now, such measurements exist only for a few 100 stars in the region where the stellar surface density transitions from being dominated by the NSC into being dominated by the NSD. We want to study the stellar population from the centre of the NSC out to well beyond its effective radius, where the NSD dominates. In this way, we can investigate whether and how the mean properties and kinematics of the stars change systematically. We conducted spectroscopic observations with Flamingos-2 in the $K$ -band via a continuous slit-scan. The data extend from the central NSC into the inner NSD, out to $\pm$ 32 pc from $\sgra$ along Galactic longitude $l$ .Based on their CO equivalent width we classify the stars as hot or cool stars. The former are massive, young stars, while almost all of the latter are older than one to a few gigayears. Applying full-spectral fitting, we measure the overall metallicity $\mh$ and line-of-sight velocity $\vlos$ for \textgreater 2 500 cool stars, increasingexisting samples outside of the very centre by a factor of 3 in terms of number of stars, and  by more than an order of magnitude in terms of covered area.We present the first continuous spatial maps and profiles of the mean value of various stellar and kinematic parameters. We identify hot, young stars across the field of view. Some stars appear to be isolated from other hot stars, while others accumulate within 2.7 pc of the Quintuplet cluster, or the central parsec cluster.The position-velocity curve of the cool stars shows no dependence on $\mh$ , but it depends on the colour of the stars. The colour may be a tracer of the line-of-sight distance, and thus distinguish stars located in the NSC from those in the NSD. A subset of the cool stars has high velocities \textgreater 150 $\kms$ , they may be associated with the bar or tidal tails of star clusters.

</div>

<div id="div_fig1">

<img src="tmp_2503.11856/./mhprofile_running_noex_cutnorth_cutblue.png" alt="Fig7.1" width="33%"/><img src="tmp_2503.11856/./lmetprof_running_noex_cutnorth_cutblue.png" alt="Fig7.2" width="33%"/><img src="tmp_2503.11856/./mh_fu_hk_noex.png" alt="Fig7.3" width="33%"/>

**Figure 7. -** Moving average $\mh$ profile of Galactic Centre late-type stars. Top panel: Profile along Galactic longitude, centred on $\sgra$. Red lines denote the average over the entire FOV,  blue dashed lines for stars at $b$\textless 1 pc, i.e. without stars located in the northern region of the FOV (shown as orange dot-dashed line), and black dot-dashed line excludes, in addition, stars with $\col$\textless1.7 mag. The average is computed on 200 stars, uncertainties are Poisson errors. The green line denotes the moving average of $\mh$ of 100 stars from \cite{2017MNRAS.464..194F,2020MNRAS.494..396F}.
 Vertical dashed lines denote the NSC $\re$=5 pc, and the centre at 0 pc. Middle panel: Same as top panel, but including NSD data from \cite{2021A&A...649A..83F}, shown as purple dashed and solid orange line with diamond symbols,  and \cite{2022MNRAS.513.5920F}, shown together with the \cite{2017MNRAS.464..194F,2020MNRAS.494..396F} data as green square symbols. To aid visibility, we plot error bars only every $\sim$20 pc. Bottom panel: Profile along observed $\col$, for different regions in our FOV, and averaged over 50--100 stars, to account for the lower and different number of stars per field.
 (*fig:lspav*)

</div>
<div id="div_fig2">

<img src="tmp_2503.11856/./SPar_bin240__noex1.png" alt="Fig19" width="100%"/>

**Figure 19. -** Mean effective temperature $\teff$, overall metallicity $\mh$, and surface gravity $\logg$. The regions with fainter stars in Fig. $\re$f{fig:vorK}(top panel) have higher $\teff$\space and $\logg$, as expected. High-velocity stars and foreground stars were excluded. Each bin contains $\sim$40 stars. (*fig:vorSP*)

</div>
<div id="div_fig3">

<img src="tmp_2503.11856/./Kcol-Ak_bin40__noex1.png" alt="Fig18" width="100%"/>

**Figure 18. -** Mean photometric properties in different Voronoi bins, top: $K_{S,0}$, middle $\col$, bottom $A_{K_{S,0}}$. The mean $K_{S,0}$ indicates that the stars in the central region are slightly fainter than stars in the outer regions, especially in the west. The stars in the centre are slightly redder (higher mean $\col$), and in this region, the extinction $\ak$ is higher. High-velocity stars and foreground stars were excluded. Each bin contains $\sim$40 stars.  (*fig:vorK*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.11856"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{\textcolor{red}{[#1]}}$
$\newcommand{\edit}[1]{\textcolor{red}{#1}}$</div>



<div id="title">

# Advancing European High-Contrast Imaging R\&D Towards the Habitable Worlds Observatory

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2503.12707-b31b1b.svg)](https://arxiv.org/abs/2503.12707)<mark>Appeared on: 2025-03-18</mark> -  _Accepted for publication in Astrophysics and Space Science_

</div>
<div id="authors">

I. Laginja, et al. -- incl., <mark>E. Matthews</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>O. Krause</mark>

</div>
<div id="abstract">

**Abstract:** The Habitable Worlds Observatory (HWO) will enable a transformative leap in the direct imaging and characterization of Earth-like exoplanets. For this, NASA is focusing on early investment in technology development prior to mission definition and actively seeking international partnerships earlier than for previous missions. The "R \& D for Space-Based HCI in Europe" workshop, held in March 2024 at Paris Observatory, convened leading experts in high-contrast imaging (HCI) to discuss European expertise and explore potential strategies for European contributions to HWO. This paper synthesizes the discussions and outcomes of the workshop, highlighting Europe's critical contributions to past and current HCI efforts, the synergies between ground- and space-based technologies, and the importance of laboratory testbeds and collaborative funding mechanisms.Key conclusions include the need for Europe to invest in technology development for areas such as deformable mirrors and advanced detectors, and establish or enhance laboratory facilities for system-level testing. Putting emphasis on the urgency of aligning with the timeline of the HWO, the participants called on an open affirmation by the European Space Agency (ESA) that a European contribution to HWO is clearly anticipated, to signal national agencies and unlock funding opportunities at the national level. Based on the expertise demonstrated through R \& D, Europe is poised to play a pivotal role in advancing global HCI capabilities, contributing to the characterization of temperate exoplanets and fostering innovation across domains.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2503.12707"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

367  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

3  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
